# Imports

In [ ]:
# required imports
from firebolt.service.manager import ResourceManager
from firebolt.client import DEFAULT_API_URL
from firebolt.client.auth import ClientCredentials

# use debug function for pretty printing
from devtools import debug

# Logging

In [ ]:
# configure logging
import logging

logging.basicConfig(
    format="{asctime} - {name} - {levelname} - {message}", style="{", level="INFO"
)

# uncomment to show every web request
# logging.getLogger("firebolt.client").setLevel("DEBUG")

# uncomment to suppress info messages from core database/engine operations
# logging.getLogger("firebolt.service").setLevel("WARNING")
# logging.getLogger("firebolt.model").setLevel("WARNING")

# uncomment to suppress info messages from running queries
# logging.getLogger("firebolt.db").setLevel("WARNING")

# Initialize ResourceManager

In [ ]:
client_id = ""
client_secret = ""
account_name = ""
api_endpoint = DEFAULT_API_URL  # Optional

In [ ]:
rm = ResourceManager(
    auth=ClientCredentials(client_id, client_secret),
    account_name=account_name,
    api_endpoint=api_endpoint,
)

Specify a database_name and engine_name below if you want, or run as-is to use the defaults.

In [ ]:
import time

default_name = f"temp_{int(time.time())}"
database_name = default_name  # Change to use you own name for creating a new database
engine_name = default_name  # Change to use you own name for creating a new engine

# Database

### Create database

In [ ]:
database = rm.databases.create(name=database_name, region="us-east-1")
debug(database)

### Get by name

In [ ]:
database = rm.databases.get(database_name)
debug(database)

### Get engines attached to a database

In [ ]:
engines = database.get_attached_engines()

debug(engines)

# Engine

### Create engine

In [ ]:
engine = rm.engines.create(name=engine_name)
debug(engine)

### Get by name

In [ ]:
engine = rm.engines.get(engine_name)
debug(engine)

### Attach to database

In [ ]:
# Attaching an already attached engine will raise an OperationalError
if not engine.database:
    engine.attach_to_database(database=rm.databases.get(database_name))

debug(engine.database)

### Start

In [ ]:
engine = engine.start()
debug(engine)

### Run a query

In [ ]:
with engine.get_connection() as connection:
    with connection.cursor() as cursor:
        result = cursor.execute("select 1")
        debug(result)

### Stop

In [ ]:
stopped_engine = engine.stop()
debug(stopped_engine)

### Get an engine's database

In [ ]:
debug(engine.database)

# Instance Types

### Get an instance type by name

In [ ]:
instance_type = rm.instance_types.get("B2")
debug(instance_type)

### Get cheapest available instance type

In [ ]:
cheapest_instance_type = rm.instance_types.cheapest_instance
debug(cheapest_instance_type)